In [ ]:
# Demo de Integración - Patrones de Diseño y SQL Avanzado

Este notebook demuestra la implementación y uso de los patrones de diseño en el sistema de gestión de ventas:

1. **Singleton Pattern** - Conexión única a la base de datos
2. **Factory Method Pattern** - Creación de modelos de datos
3. **Builder Pattern** - Construcción fluida de consultas SQL
4. **Decorator Pattern** - Funcionalidades adicionales (logging, timing, caching)
5. **Strategy Pattern** - Diferentes algoritmos de análisis


In [ ]:
# Importaciones necesarias
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
from datetime import datetime

# Importar clases del proyecto
from src.database.connection import DatabaseConnection
from src.database.query_examples import QueryExamples
from src.utils.model_factory import ModelFactoryRegistry, CategoryFactory, ProductFactory
from src.utils.query_builder import SQLQueryBuilder, QueryBuilderDirector
from src.utils.decorators import database_operation, timing_decorator, logging_decorator
from src.utils.analysis_strategies import (
    RevenueAnalysisStrategy, 
    QuantityAnalysisStrategy, 
    CustomerBehaviorAnalysisStrategy,
    SalesAnalysisContext,
    AnalysisStrategyFactory
)

print("✅ Importaciones completadas exitosamente")
